In [1]:
!python -V

Python 3.9.21


In [2]:
import pandas as pd

In [3]:
import pickle

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import root_mean_squared_error

In [6]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment_tracking/data/mlruns/1', creation_time=1748196092546, experiment_id='1', last_update_time=1748196092546, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [7]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [8]:
df_train = read_dataframe('/workspaces/mlops-zoomcamp/02-experiment_tracking/data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('/workspaces/mlops-zoomcamp/02-experiment_tracking/data/green_tripdata_2021-02.parquet')

In [9]:
len(df_train), len(df_val)

(73908, 61921)

In [10]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [11]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [12]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [ ]:
with mlflow.start_run():
    mlflow.set_tag("model", "linear-regression")
    lr = LinearRegression()
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)

    root_mean_squared_error(y_val, y_pred)

with mlflow.start_run():

    mlflow.set_tag("developer", "yashwanth")

    mlflow.log_param("train-data-path", "./workspaces/mlops-zoomcamp/02-experiment_tracking/data/green_tripdata_2021-01.parquet")
    mlflow.log_param("val-data-path", "./workspaces/mlops-zoomcamp/02-experiment_tracking/data/green_tripdata_2021-02.parquet")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)

    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path='/workspaces/mlops-zoomcamp/02-experiment_tracking/models/lin_reg.bin', artifact_path='models')

7.758715200888857

In [15]:
with open('/workspaces/mlops-zoomcamp/02-experiment_tracking/models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [ ]:
with mlflow.start_run():

    mlflow.set_tag("developer", "yashwanth")

    mlflow.log_param("train-data-path", "./workspaces/mlops-zoomcamp/02-experiment_tracking/data/green_tripdata_2021-01.parquet")
    mlflow.log_param("val-data-path", "./workspaces/mlops-zoomcamp/02-experiment_tracking/data/green_tripdata_2021-02.parquet")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)

    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path='/workspaces/mlops-zoomcamp/02-experiment_tracking/models/lin_reg.bin', artifact_path='models')

In [14]:
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [15]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [16]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [17]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [21:34:21] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:9.83098                           
[1]	validation-rmse:8.39753                           
[2]	validation-rmse:7.56812                           
[3]	validation-rmse:7.10178                           
[4]	validation-rmse:6.84290                           
[5]	validation-rmse:6.69214                           
[6]	validation-rmse:6.60732                           
[7]	validation-rmse:6.55317                           
[8]	validation-rmse:6.51521                           
[9]	validation-rmse:6.49119                           
[10]	validation-rmse:6.47270                          
[11]	validation-rmse:6.46047                          
[12]	validation-rmse:6.45092                          
[13]	validation-rmse:6.44517                          
[14]	validation-rmse:6.44003                          
[15]	validation-rmse:6.43773                          
[16]	validation-rmse:6.43413                          
[17]	validation-rmse:6.43062                          
[18]	valid

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [21:35:45] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:9.11142                                                       
[1]	validation-rmse:7.65840                                                       
[2]	validation-rmse:7.02778                                                       
[3]	validation-rmse:6.74361                                                       
[4]	validation-rmse:6.61862                                                       
[5]	validation-rmse:6.55392                                                       
[6]	validation-rmse:6.51770                                                       
[7]	validation-rmse:6.49495                                                       
[8]	validation-rmse:6.48037                                                       
[9]	validation-rmse:6.47060                                                       
[10]	validation-rmse:6.46542                                                      
[11]	validation-rmse:6.46179                                                      
[12]

...

[3]	validation-rmse:10.52570                                                      
[4]	validation-rmse:10.18762                                                      
[5]	validation-rmse:9.87738                                                       
[6]	validation-rmse:9.59533                                                       
[7]	validation-rmse:9.33661                                                       
[8]	validation-rmse:9.10193                                                       
[9]	validation-rmse:8.88689                                                       
[10]	validation-rmse:8.69265                                                      
[11]	validation-rmse:8.51596                                                      
[12]	validation-rmse:8.35617                                                      
[13]	validation-rmse:8.21145                                                      
[14]	validation-rmse:8.07918                                                      
[15]

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [23:11:18] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:10.89320                                                      
[1]	validation-rmse:9.85566                                                       
[2]	validation-rmse:9.04961                                                       
[3]	validation-rmse:8.42960                                                       
[4]	validation-rmse:7.95950                                                       
[5]	validation-rmse:7.60332                                                       
[6]	validation-rmse:7.33726                                                       
[7]	validation-rmse:7.13774                                                       
[8]	validation-rmse:6.98822                                                       
[9]	validation-rmse:6.87502                                                       
[10]	validation-rmse:6.78857                                                      
[11]	validation-rmse:6.72536                                                      
[12]

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [23:14:05] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:11.14288                                                      
[1]	validation-rmse:10.26106                                                      
[2]	validation-rmse:9.54477                                                       
[3]	validation-rmse:8.95511                                                       
[4]	validation-rmse:8.48697                                                       
[5]	validation-rmse:8.09783                                                       
[6]	validation-rmse:7.79440                                                       
[7]	validation-rmse:7.55289                                                       
[8]	validation-rmse:7.35472                                                       
[9]	validation-rmse:7.19946                                                       
[10]	validation-rmse:7.07998                                                      
[11]	validation-rmse:6.98859                                                      
[12]

In [ ]:
params = {
    'max_depth': 33,
    'learning_rate': 0.19087733393153664,
    'reg_alpha': 0.34241467579506146,
    'reg_lambda': 0.07513705733067294,
    'min_child_weight': 1.1849866953729065,
    'objective': 'reg:linear',
    'seed': 42
}

mlflow.xgboost.autolog(
    log_datasets=False,  # Disables dataset logging
)

booster = xgb.train(
    params=params,
    dtrain=train,
    num_boost_round=1000,
    evals=[(valid, 'validation')],
    early_stopping_rounds=50
)

2025/05/26 00:20:22 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '0130c660e8c44ff9917ec79b0fd6df12', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow
/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [00:20:22] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:10.70532
[1]	validation-rmse:9.57176
[2]	validation-rmse:8.73389
[3]	validation-rmse:8.12269
[4]	validation-rmse:7.68665
[5]	validation-rmse:7.37072
[6]	validation-rmse:7.14364
[7]	validation-rmse:6.98226
[8]	validation-rmse:6.86682
[9]	validation-rmse:6.78111
[10]	validation-rmse:6.71739
[11]	validation-rmse:6.66942
[12]	validation-rmse:6.63434
[13]	validation-rmse:6.60555
[14]	validation-rmse:6.58467
[15]	validation-rmse:6.56804
[16]	validation-rmse:6.55394
[17]	validation-rmse:6.54357
[18]	validation-rmse:6.53343
[19]	validation-rmse:6.52578
[20]	validation-rmse:6.51985
[21]	validation-rmse:6.51611
[22]	validation-rmse:6.51327
[23]	validation-rmse:6.51070
[24]	validation-rmse:6.50946
[25]	validation-rmse:6.50732
[26]	validation-rmse:6.50592
[27]	validation-rmse:6.50471
[28]	validation-rmse:6.50265
[29]	validation-rmse:6.49977
[30]	validation-rmse:6.49766
[31]	validation-rmse:6.49648
[32]	validation-rmse:6.49490
[33]	validation-rmse:6.49261
[34]	validation-rmse:6.

2025/05/26 00:23:23 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2025/05/26 00:23:23 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [00:23:23] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."
2025/05/26 00:23:28 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [17]:
mlflow.xgboost.autolog(disable=True)

In [18]:
with mlflow.start_run():

    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
    'max_depth': 33,
    'learning_rate': 0.19087733393153664,
    'reg_alpha': 0.34241467579506146,
    'reg_lambda': 0.07513705733067294,
    'min_child_weight': 1.1849866953729065,
    'objective': 'reg:linear',
    'seed': 42
    }
    
    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    with open('/workspaces/mlops-zoomcamp/02-experiment_tracking/models/preprocessor.b', 'wb') as f_out:
        pickle.dump(dv, f_out)

    mlflow.log_artifact(local_path='/workspaces/mlops-zoomcamp/02-experiment_tracking/models/preprocessor.b', artifact_path='preprocessor')

    mlflow.xgboost.log_model(booster, artifact_path='models_mlflow')

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [17:34:41] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:10.70532
[1]	validation-rmse:9.57176
[2]	validation-rmse:8.73389
[3]	validation-rmse:8.12269
[4]	validation-rmse:7.68665
[5]	validation-rmse:7.37072
[6]	validation-rmse:7.14364
[7]	validation-rmse:6.98226
[8]	validation-rmse:6.86682
[9]	validation-rmse:6.78111
[10]	validation-rmse:6.71739
[11]	validation-rmse:6.66942
[12]	validation-rmse:6.63434
[13]	validation-rmse:6.60555
[14]	validation-rmse:6.58467
[15]	validation-rmse:6.56804
[16]	validation-rmse:6.55394
[17]	validation-rmse:6.54357
[18]	validation-rmse:6.53343
[19]	validation-rmse:6.52578
[20]	validation-rmse:6.51985
[21]	validation-rmse:6.51611
[22]	validation-rmse:6.51327
[23]	validation-rmse:6.51070
[24]	validation-rmse:6.50946
[25]	validation-rmse:6.50732
[26]	validation-rmse:6.50592
[27]	validation-rmse:6.50471
[28]	validation-rmse:6.50265
[29]	validation-rmse:6.49977
[30]	validation-rmse:6.49766
[31]	validation-rmse:6.49648
[32]	validation-rmse:6.49490
[33]	validation-rmse:6.49261
[34]	validation-rmse:6.

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [17:36:01] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
2025/05/27 17:36:05 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [19]:
logged_model = 'runs:/e15338177cdd473ba8dd2905d5faedf6/models_mlflow'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

In [20]:
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: models_mlflow
  flavor: mlflow.xgboost
  run_id: e15338177cdd473ba8dd2905d5faedf6

In [21]:
xgboost_model = mlflow.xgboost.load_model(logged_model)

In [22]:
xgboost_model

In [23]:
y_pred = xgboost_model.predict(valid)

In [29]:
y_pred[:10]

array([14.379493 ,  6.9471116, 14.944981 , 24.485113 ,  9.498932 ,
       17.228973 , 11.037789 ,  8.163006 ,  9.922154 , 19.589582 ],
      dtype=float32)